In [1]:
!pip install transformers accelerate datasets torch torchvision peft pillow

In [25]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType, PeftConfig, PeftModel
from transformers import AutoModelForCausalLM
import torch
from transformers import TrainingArguments, Trainer

In [3]:
#ask_llm = pipeline(
 #   model = "Qwen/Qwen2.5-3B-Instruct",
  #  device = "cuda"
#)

In [4]:
#print(ask_llm("who is ali daei")[0]["generated_text"])

In [5]:
#print(ask_llm("هوش مصنوعی چیست؟")[0]["generated_text"])

In [6]:
row_data = load_dataset("SajjadAyoubi/persian_qa")

In [7]:
row_data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [8]:
raw_data = row_data.remove_columns(["id", "title", "answers"])

In [9]:
raw_data

DatasetDict({
    train: Dataset({
        features: ['context', 'question'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['context', 'question'],
        num_rows: 930
    })
})

In [10]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

def preprocess(sample):
    sample = sample["question"] + "\n" + sample["context"]
    tokenized = tokenizer(
        sample,
        max_length = 128,
        truncation = True,
        padding = "max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [11]:
data = raw_data.map(preprocess)

In [12]:
data

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['context', 'question', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 930
    })
})

In [13]:
print(data["train"][0])

{'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.', 'question': 'شرکت فولاد مبارکه در کجا واقع شده است', 'inp

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map = "cuda",
    torch_dtype = torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

In [16]:
model = get_peft_model(model, lora_config)

In [17]:
training_args = TrainingArguments(
    num_train_epochs = 2,
    learning_rate = 0.001,
    logging_steps = 100,
    report_to = "tensorboard"
)

In [18]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = data["train"]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [19]:
print("Start Training...")
trainer.train()
print("Training finished...")

Start Training...


Step,Training Loss
100,1.670800
200,1.538500
300,1.487300
400,1.410400
500,1.331200
600,1.261900
700,1.178400
800,1.116200
900,1.050100
1000,1.012200


Training finished...


In [20]:
trainer.save_model("/kaggle/working/")
tokenizer.save_pretrained("/kaggle/working/")

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/chat_template.jinja',
 '/kaggle/working/vocab.json',
 '/kaggle/working/merges.txt',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

In [21]:
path = "/kaggle/working/"

In [26]:
config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code = True)
model = PeftModel.from_pretrained(base, path)
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
inputs = tokenizer("شرکت فولاد مبارکه در کجا واقع شده است", return_tensors = "pt").to(model.device)
output = model.generate(
    input_ids = inputs["input_ids"],
    attention_mask = inputs["attention_mask"],
    max_new_tokens=100
)

print(tokenizer.decode(output[0]))

شرکت فولاد مبارکه در کجا واقع شده است؟
شرکت فولاد مبارکه یکی از بزرگ‌ترین شرکت‌های فولاد در خاورمیانه و ایران است. این شرکت در سال ۱۳۵۰ به‌عنوان شرکت زندهٔ اول ایران در حوزهٔ فولاد تأسیس شد و در ۲


In [29]:
def generate_response(query):
    inputs = tokenizer(query, return_tensors = "pt").to(model.device)
    output = model.generate(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        max_new_tokens = 256
    )
    return(tokenizer.decode(output[0]))

In [31]:
print(generate_response("به چی میگن ربات؟"))

به چی میگن ربات؟
رَبَط (به انگلیسی: Link) یک مجموعهٔ تلویزیونی فیلمی دستaurus هیلز است که در سال ۲۰۱۲ و در شصت و نخست و دوم سال سالنیکو برای خانه مردم اکستريم داستان اولین بار به طور آنلاین پخش شد. ربط از نویسندگی حیوان‌شناس زن دیوانه‌برداری داستان فلمی است که «زندراف» نوشتهٔ هیلز در سال ۱۹۸۷ بود. ربط از ۵۰۰۰ سالگی یک مجموعه تلویزیونی داستانی از یک فرد با نام داستان‌پرداز شهرت‌مند مجموعه است که در یک شهر تلویزیونی نوشته شده‌


In [32]:
print(generate_response("بزرگ ترین مجموعه تولید فولاد ایران چیست؟"))

بزرگ ترین مجموعه تولید فولاد ایران چیست؟
فولاد، نام دارویی برای بیماری پنج‌گوش است. فولاد دارویی است که در شکایتی از گریم‌ها و خرسنکا به کار می‌رود. در تاکیدی هنگام استفاده از فولاد، از لحاظ پیشرفته، یک نوع دارویی برای بیماری پنج‌گوش است. پندار بیماری پنج‌گوش نیز یک نوع از جمله‌معانی خون‌شویی و انواع مختلف از گریم و فuktار و … به‌شمار می‌رود. بیماری پنج‌گوش، قدرت گفتمانی کاهش دهنده و صدایی ناشی از مشکل در پندار است. بیماری پنج‌گوش در بیشتر مردم وجود ندارد که از آن علائمی که دارد به ع


In [33]:
print(generate_response("تعریف پریانت چیست؟"))

تعریف پریانت چیست؟
پریانت نوعی نوشیدنی جهان‌گستری است که از گوشت پستاندار‌ها به عنوان مادهٔ ناهzung و یا ته冶 شده در بوده و نیز از آب و خوراک ضرب‌بالهٔ خانگی به عنوان مادهٔ نشسته به شکل یک گوشت‌ساز (به انگلیسی : Steakhouse ) انتخاب می‌شود. هر گونه نوشیدنی گوستی شامل ضرب‌بالهٔ گوشت (با ترکیب گوسفند، عجیر، دانش یا بعوض) است و ممکن است با آب و غذای ضربگونه جمع‌آوری شود. پریانت از گوشت ضرب‌باله شده فرم‌بندی شده و سپس دانه گوشت در آن به زمان برقرار
